# Modelos Asignación Médicos Domiciliarios

### Indicadores a medir:
- Distancia total recorrida por el médico
- Cantidad de solicitudes atendidas por cada médico
- Tiempo muerto de cada médico
- Tiempo de espera de cada solicitud: tiempo desde que el paciente solicita la cita hasta que el médico la atiende

### Instancias: 30/05/2022 - 05/06/2022

- Se filtra por FECHA INICIO y FECHA SOLICITUD
- La llegada corresponde a la llegada de la solicitud
- Cuando la duración son valores inferiores a 10 se toman como datos atípicos y se cambia el valor por la media de la duración de las solicitudes
- Cuando pasan más de 3 horas desde que se hizo la solicitud hasta que se atiende, se toma como que no se atendió la solicitud y se elimina
- Es necesario ajustar los médicos asignados a cada día de las instancias, por lo que se adaptaron a la distribución actual llenando las franjas horarias con menos médicos
- Los ID de los médicos corresponden al nombre, sin embargo, puede que la franja horaria sea diferente

## Modelo Final - Heurística de asignación de médicos

### Leer los datos de excel

In [ ]:
import pandas as pd

#Funcion para leer los datos de los pacientes
def leer_datos():
    # Leer el archivo de excel
    df = pd.read_excel('Instancias.xlsx', sheet_name='Pacientes7') #!!!! Se cambia el nombre de la hoja
    df1 = pd.read_excel("Instancias.xlsx", sheet_name="Medicos7") #!!!! Se cambia el nombre de la hoja
    df3 = pd.read_excel("Instancias.xlsx", sheet_name="Clientesv4")
    
    # Crear un diccionario para almacenar los datos de los pacientes y otro para los medicos
    pacientesDict = {}
    medicos = {}
    Clientes = {}
    
    # diccionario con los tipos de cliente y el tiempo de atencion esperado
    for index, row in df3.iterrows():
        llave = row["Tipo"]
        valor = row["Tiempo"]
        Clientes[llave] = valor


    # Iterar sobre las filas del DataFrame
    for index, row in df.iterrows():
        llave = row["ID Servicio"]
        valor1 = row["Llegada"]
        valor2 = row["Atencion"]
        valor3 = row["Latitud"]
        valor4 = row["Longitud"]
        valor5 = row["Entidad"]

        # Agregar valores a la lista correspondiente en el diccionario
        if llave in pacientesDict:
            pacientesDict[llave].append([valor1, valor2, valor3, valor4, valor5])
        else:
            pacientesDict[llave] = [int(valor1), int(valor2), valor3, valor4, str(valor5)]
    pacientes = pacientesDict

    # Iterar sobre las filas del DataFrame
    for index, row in df1.iterrows():
        llave = row["ID Medico"]
        Disponibiidad = 0 
        UltOcupado = 0
        NumSol = 0
        lat = 4.681230925773865
        lon = -74.06200456196707
        Inicioj = row["Inicio Jornada"]
        Finj = row["Fin Jornada"]
    
        # Agregar valores a la lista correspondiente en el diccionario
        if llave in medicos:
            medicos[llave].append([Disponibiidad, UltOcupado, lat, lon, NumSol, Inicioj, Finj])
        else:
            medicos[llave] = [int(Disponibiidad), int(UltOcupado),lat, lon, NumSol, int(Inicioj), int(Finj)]
    doctores = medicos
    
    return doctores, pacientes, Clientes

doctores = leer_datos()[0]
pacientes = leer_datos()[1]
Clientes = leer_datos()[2]

### Calcular la distancia/tiempo entre cada par de nodos

In [ ]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

### Cambiar datos para Gantt

In [ ]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

### Regla de Prioridad

In [ ]:
# Regla de prioridad
def prioridad(NumSolicitudes, Tdesplazamiento, Mdescoupa):
    # Si el número de solicitudes esta entre 0 y 2
    if NumSolicitudes >= 0 and NumSolicitudes <= 2:
        beta = 0.55
        alpha = 0.45
    # Si el número de solicitudes esta entre 3 y 4
    elif NumSolicitudes >= 3 and NumSolicitudes <= 4:
        beta = 0.65
        alpha = 0.35
    # Si el número de solicitudes esta entre 5 o más
    elif NumSolicitudes >= 5:
        beta = 0.75
        alpha = 0.25

    # Calculamos la prioridad
    pesoPrioridad = NumSolicitudes * alpha + Tdesplazamiento + Mdescoupa * beta

    return pesoPrioridad


### Asignación de los médicos

In [ ]:
global doctores

import pandas as pd

# Función para asignar doctor a paciente
def asignar_doctor(atencion,latitud, longitud, minuto, gantt, IDservicio, TipoCliente):

    # Lista de posible asignacion
    posibleAsignacion = []

    # Actualizar estado de los médicos
    if minuto > 0:
        for i in doctores:
            # Si el minuto actual es mayor al ultimo ocupado significa que el médico esta disponible
            if minuto > doctores[i][1]:
                # Actualizar estado del médico
                doctores[i][0] = 0
    
    for i in doctores:
        # Si el médico esta en su jornada laboral y esta disponible
        if minuto >= doctores[i][5] and minuto <= doctores[i][6] and doctores[i][0] == 0:
            # Agregar médico y su distancia a la lista de posible asignacion

            # Encontrar distancia de los médicos al paciente
            distancia = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
            # Agregar médico y su distancia a la lista de posible asignacion
            posibleAsignacion.append([i, distancia])
    
    # Si no hay ningún médico disponible
    if len(posibleAsignacion) == 0:
        # Se buscan médicos que esten en su jornada laboral para asignarles el paciente usando la regla de prioridad

        for i in doctores:
            # Si el doctor esta en su jornada laboral
            if minuto >= doctores[i][5] and minuto <= doctores[i][6]:
                # Se aplican las reglas de prioridad

                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                # Calcular tiempo de desocupacion
                Mdesocupa = doctores[i][1]
                # Calcular prioridad
                pesoPrioridad = prioridad(doctores[i][4], Tdesplazamiento, Mdesocupa)
                # Agregar a la lista de posible asignacion
                posibleAsignacion.append([i, pesoPrioridad])
        

        # Si existen médicos en su jornada laboral
        if len(posibleAsignacion) > 0:
            # Escoger el médico con mayor prioridad, es decir, el que tenga menor peso

            # Filtrar a los médicos que cumplan con el tiempo de servicio
            for i in posibleAsignacion:
                # calcular el desplazamiento del medico
                Tdesplazamiento = distancia_tierra(doctores[i[0]][2], latitud, doctores[i[0]][3], longitud)

                # Si el médico no cumple con el tiempo de servicio se elimina de la lista de posible asignacion
                if minuto + Tdesplazamiento > minuto + Clientes[TipoCliente]:
                    #crear lista de médicos que no cumplen con el tiempo de servicio
                    listaMedicosNoCumplen = []
                    #agregar médicos que no cumplen con el tiempo de servicio
                    listaMedicosNoCumplen.append(i[0])
                    #eliminar médicos que no cumplen con el tiempo de servicio
                    posibleAsignacion.remove(i)

            # si existen médicos que cumplan con el tiempo de servicio
            if len(posibleAsignacion) > 0:
                # Ordenar la lista de posible asignacion
                posibleAsignacion.sort(key=lambda x: x[1])
                # Asignar el doctor con menor prioridad
                doc_Asignar = posibleAsignacion[0]

                # Actualizar variables del médico asignado
                Mdesocupa = doctores[doc_Asignar[0]][1]
                Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

                # Actualizar el gantt
                gantt.append({'Solicitud': IDservicio, 'Medico': doc_Asignar[0], 'Inicio': convertMin(Mdesocupa + Tdesplazamiento), 'Fin': convertMin(Mdesocupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Inicio traslado': Mdesocupa, 'Fin traslado': Mdesocupa + Tdesplazamiento, 'Inicio atencion': Mdesocupa + Tdesplazamiento, 'Fin atencion': Mdesocupa + Tdesplazamiento + atencion, 'Llegada solicitud': minuto, 'Tiempo contratado': Clientes[TipoCliente], 'Tipo de asignacion': 'Tipo 1'})
                
                # Actualizar estado del médico
                doctores[doc_Asignar[0]][0] = 1
                doctores[doc_Asignar[0]][1] = Mdesocupa + Tdesplazamiento + atencion
                doctores[doc_Asignar[0]][2] = latitud
                doctores[doc_Asignar[0]][3] = longitud
                doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1

            # Si no existen médicos que cumplan con el tiempo de servicio
            else:
                # Calcular el médico que atienda la solicitud más rapido
                for i in listaMedicosNoCumplen:
                    # Calcular tiempo de desplazamiento
                    Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                    # Calcular tiempo de desocupacion
                    Mdesocupa = doctores[i][1]
                    # Cambiar el peso de prioridad por el tiempo de desplazamiento
                    listaMedicosNoCumplen[i][1] = Mdesocupa + Tdesplazamiento

                # Ordenar la lista de posible asignacion
                listaMedicosNoCumplen.sort(key=lambda x: x[1])

                # Asignar el médico con menor tiempo de desplazamiento
                doc_Asignar = listaMedicosNoCumplen[0]

                # Actualizar variables del médico asignado
                Mdesocupa = doctores[doc_Asignar[0]][1]
                Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

                # Actualizar el gantt
                gantt.append({'Solicitud': IDservicio, 'Medico': doc_Asignar[0], 'Inicio': convertMin(Mdesocupa + Tdesplazamiento), 'Fin': convertMin(Mdesocupa + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Inicio traslado': Mdesocupa, 'Fin traslado': Mdesocupa + Tdesplazamiento, 'Inicio atencion': Mdesocupa + Tdesplazamiento, 'Fin atencion': Mdesocupa + Tdesplazamiento + atencion, 'Llegada solicitud': minuto, 'Tiempo contratado': Clientes[TipoCliente], 'Tipo de asignacion': 'Tipo 2'})

                # Actualizar estado del médico
                doctores[doc_Asignar[0]][0] = 1
                doctores[doc_Asignar[0]][1] = Mdesocupa + Tdesplazamiento + atencion
                doctores[doc_Asignar[0]][2] = latitud
                doctores[doc_Asignar[0]][3] = longitud
                doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1
        else:
            # No hay médicos disponibles
            print("No hay médicos disponibles")
                
    # Si hay uno o más médicos disponibles
    elif len(posibleAsignacion) > 0:
        # Se filtrarán los médicos que no cumplan con el tiempo de servicio
        for i in posibleAsignacion:
            # Si el médico no cumple con el tiempo de servicio se elimina de la lista de posible asignacion
            if minuto + i[1] > minuto + Clientes[TipoCliente]:
                #crear lista de médicos que no cumplen con el tiempo de servicio
                listaMedicosNoCumplen = []
                #agregar médicos que no cumplen con el tiempo de servicio
                listaMedicosNoCumplen.append(i[0])
                #eliminar médicos que no cumplen con el tiempo de servicio
                posibleAsignacion.remove(i)
        
        # Si existen médicos que cumplan con el tiempo de servicio
        if len(posibleAsignacion) > 0:

            # calcular el peso de prioridad de cada médico
            for i in posibleAsignacion:
                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i[0]][2], latitud, doctores[i[0]][3], longitud)
                # Calcular tiempo de desocupacion
                Mdesocupa = doctores[i[0]][1]
                # Calcular prioridad
                pesoPrioridad = prioridad(doctores[i[0]][4], Tdesplazamiento, Mdesocupa)

                # Cambiar el tiempo de desplazamiento por el peso de prioridad
                i[1] = pesoPrioridad
            
            # Ordenar la lista de posible asignacion
            posibleAsignacion.sort(key=lambda x: x[1])

            # Asignar el médico con menor prioridad
            doc_Asignar = posibleAsignacion[0]
            # Actualizar variables del médico asignado
            Mdesocupa = doctores[doc_Asignar[0]][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Medico': doc_Asignar[0], 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Inicio traslado': minuto, 'Fin traslado': minuto + Tdesplazamiento, 'Inicio atencion': minuto + Tdesplazamiento, 'Fin atencion': minuto + Tdesplazamiento + atencion, 'Llegada solicitud': minuto, 'Tiempo contratado': Clientes[TipoCliente], 'Tipo de asignacion': 'Tipo 3'})
            # Actualizar estado del médico
            doctores[doc_Asignar[0]][0] = 1
            doctores[doc_Asignar[0]][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar[0]][2] = latitud
            doctores[doc_Asignar[0]][3] = longitud
            doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1
        
        # Si no existen médicos que cumplan con el tiempo de servicio
        else:
            # Se asignará el médico con menor tiempo de desplazamiento
            print(listaMedicosNoCumplen)
            for i in listaMedicosNoCumplen:
                # Calcular tiempo de desplazamiento
                Tdesplazamiento = distancia_tierra(doctores[i][2], latitud, doctores[i][3], longitud)
                # Cambiar el peso de prioridad por el tiempo de desplazamiento
                listaMedicosNoCumplen[listaMedicosNoCumplen.index(i)] = [i, Tdesplazamiento]
            
            # Ordenar la lista de posible asignacion
            listaMedicosNoCumplen.sort(key=lambda x: x[1])

            # Asignar el médico con menor tiempo de desplazamiento
            doc_Asignar = listaMedicosNoCumplen[0]

            # Actualizar variables del médico asignado
            Mdesocupa = doctores[doc_Asignar[0]][1]
            Tdesplazamiento = distancia_tierra(doctores[doc_Asignar[0]][2], latitud, doctores[doc_Asignar[0]][3], longitud)

            # Actualizar el gantt
            gantt.append({'Solicitud': IDservicio, 'Medico': doc_Asignar[0], 'Inicio': convertMin(minuto + Tdesplazamiento), 'Fin': convertMin(minuto + Tdesplazamiento + atencion), 'Distancia': Tdesplazamiento, 'Inicio Jornada': doctores[doc_Asignar[0]][5], 'Fin Jornada': doctores[doc_Asignar[0]][6], 'Inicio traslado': minuto, 'Fin traslado': minuto + Tdesplazamiento, 'Inicio atencion': minuto + Tdesplazamiento, 'Fin atencion': minuto + Tdesplazamiento + atencion, 'Llegada solicitud': minuto, 'Tiempo contratado': Clientes[TipoCliente], 'Tipo de asignacion': 'Tipo 4'})

            # Actualizar estado del médico
            doctores[doc_Asignar[0]][0] = 1
            doctores[doc_Asignar[0]][1] = minuto + Tdesplazamiento + atencion
            doctores[doc_Asignar[0]][2] = latitud
            doctores[doc_Asignar[0]][3] = longitud
            doctores[doc_Asignar[0]][4] = doctores[doc_Asignar[0]][4] + 1

    return gantt


### Simular Pacientes

In [ ]:
import pandas as pd
import openpyxl

global doctores

# Función principal para asignar médicos a pacientes
def asignacion_doctor1():
    pacientes = leer_datos()[1]
    gantt = []

    # Ordenar dict de pacientes por minuto
    pacientes = dict(sorted(pacientes.items(), key=lambda item: item[1][0]))

    for i in pacientes:
        # Asignar doctor a paciente
        atencion = pacientes[i][1]
        minuto = pacientes[i][0]
        latitud = pacientes[i][2]
        longitud = pacientes[i][3]
        IDservicio = i
        TipoCliente = pacientes[i][4]
        gantt = asignar_doctor(atencion,latitud,longitud, minuto, gantt, IDservicio, TipoCliente)
    
    return gantt

In [ ]:
gantt = asignacion_doctor1()

### Gantt general

In [ ]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Medico'] = df1['Medico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Medico", color="Medico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")

fig.update_layout(xaxis_title="Tiempo", yaxis_title="Medico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()


## Modelo Optimización Lineal

In [1]:
import pandas as pd

# ---------------
# IMPORTACIÓN DE DATOS
# ---------------
df_m = pd.read_excel('Instancias.xlsx', sheet_name='Medicos7')
df_s = pd.read_excel('Instancias.xlsx', sheet_name='Pacientes7')
df_e = pd.read_excel('Instancias.xlsx', sheet_name='Clientesv5')

# ---------------
# CONJUNTOS
# ---------------
# # M = conjunto de médicos disponibles identificados por ID.
M = list(df_m['ID Medico'])
# S = conjunto de solicitudes identificadas por el ID del servicio.
S = list(df_s['ID Servicio'])
# E = conjunto de entidades.
E = list(df_e['Tipo'])

# ---------------
# PARÁMETROS
# ---------------
# Crear diccionario de parámetros
#------------ Parámetros asociados a los médicos
# i_m = documendo de identidad del médico m∈M.
i = dict(zip(M, df_m['Nombre Medico']))
# a_m = inicio de la jornada laboral del médico m∈M.
a = dict(zip(M, df_m['Inicio Jornada']))
# b_m = fin de la jornada laboral del médico m∈M.
b = dict(zip(M, df_m['Fin Jornada']))
# t_m = minuto en el que el médico m∈M está disponible.
t = dict(zip(M, df_m['Inicio Jornada']))
# c_m = número de solicitudes de servicio que se han  asignado al médico m∈M. 
c = dict(zip(M, [0]*len(M)))
lat = 4.681230925773865
lon = -74.06200456196707
# u_(i,j)^m = coordenadas i,j de la última ubicación geográfica del médico m∈M.
u = dict(zip(M, list(zip([lat]*len(M), [lon]*len(M)))))

#------------ Parámetros asociados a las solicitudes
# h_s = minuto en el que se recibió la solicitud s∈S.
h = dict(zip(S, df_s['Llegada']))
# d_s = duración de la atención de la solicitud s∈S.
d = dict(zip(S, df_s['Atencion']))
# e_s = entidad que solicitó el servicio s∈S.
e = dict(zip(S, df_s['Entidad']))
# v_(i,j)^s = coordenadas i,j de la ubicación geográfica de la solicitud s∈S.
v = dict(zip(S, list(zip(df_s['Latitud'], df_s['Longitud']))))

#------------ Parámetros asociados a las entidades
# f_e = tiempo en min. dentro del cual se debe atender las solicitudes de la entidad e∈E.
f = dict(zip(E, df_e['Tiempo']))

In [2]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

In [3]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

In [4]:
import pulp as lp

gantt = []
for s in S:

    # Tiempo máximo en el que se puede atender la solicitud s de acuerdo al tiempo contratado de la entidad.
    t_max = h[s] + f[e[s]]

    # Diccionario de distancias.
    dist = {(m, s): 0 for m in M}
    # Calcular las distancias entre los médicos y la solicitud y guardarlas en el diccionario.
    for m in M:
        distancia = distancia_tierra(v[s][0], u[m][0], v[s][1], u[m][1])
        dist[(m, s)] = distancia

    # Diccionario de pesos para balancaer la carga laboral
    p = {m: None for m in M}

    for m in M:

        if c[m] <= 2:
            p[m] = c[m] * 0.45 + (t[m] + dist[(m,s)]) * 0.55
        elif c[m] <= 4:
            p[m] = c[m] * 0.35 + (t[m] + dist[(m,s)]) * 0.65
        else:
            p[m] = c[m] * 0.25 + (t[m] + dist[(m,s)]) * 0.75

    M_disponible = list(filter(lambda m: t[m] + dist[(m,s)] <= t_max, M))
    # Solo los médicos que esten en su jornada laboral.
    M_disponible = list(filter(lambda m: t[m] + dist[(m,s)] <= b[m], M_disponible))
    
    # ---------------
    # MODELO DE OPTIMIZACIÓN
    # ---------------
    # Crear problema de optimización.
    prob = lp.LpProblem('Agendamiento_de_medicos_ADOM', lp.LpMinimize)

    # Crear variable de decisión.
    x = lp.LpVariable.dicts('x', [(s,m) for s in S for m in M_disponible], 0, 1, lp.LpBinary)

    # FUNCIÓN OBJETIVO
    prob += lp.lpSum(p[m]*x[(s,m)] for m in M_disponible)

    # RESTRICCIONES

    # 1. No se debe exceder el tiempo contratado por las entidades.    
    #for m in M:
    #    prob += t[m] + dist[(m,s)] <= t_max

    # 2. No se puede asignar una solicitud a un médico por fuera de su jornada laboral.
    #for m in M:
    #    prob += (t[m] + dist[(m,s)]) <= b[m]

    # 3. Restringir que cada solicitud sea asignada a un único médico.
    prob += lp.lpSum(x[(s,m)] for m in M_disponible) == 1

    # Optimizar el problema.
    prob.solve()

    # ---------------
    # ACTUALIZAR PARÁMETROS
    # ---------------
    print('Solicitud: ', s)
    contador = 0
    for m in M_disponible:
        if x[(s,m)].value() == 1:
            print('Se asignó médico: ', m)
            
            c[m] = c[m] + 1
            if h[s] < t[m]:
                gantt.append({'Solicitud': s , 'Medico': m, 'Inicio': convertMin(t[m] + dist[(m, s)]), 'Fin': convertMin(t[m] + dist[(m, s)] + d[s]), 'Distancia': dist[(m, s)], 'Inicio Jornada': a[m], 'Fin Jornada': b[m], 'Inicio traslado': t[m], 'Fin traslado': t[m] + dist[(m, s)], 'Inicio atencion': t[m] + dist[(m, s)], 'Fin atencion': t[m] + dist[(m, s)] + d[s], 'Tiempo contratado': f[e[s]], 'Llegada solicitud': h[s]})
                t[m] = t[m] + dist[(m,s)] + d[s]
            else:
                gantt.append({'Solicitud': s , 'Medico': m, 'Inicio': convertMin(h[s] + dist[(m, s)]), 'Fin': convertMin(h[s] + dist[(m, s)] + d[s]), 'Distancia': dist[(m, s)], 'Inicio Jornada': a[m], 'Fin Jornada': b[m], 'Inicio traslado': t[m], 'Fin traslado': h[s] + dist[(m, s)], 'Inicio atencion': h[s] + dist[(m, s)], 'Fin atencion': h[s] + dist[(m, s)] + d[s], 'Tiempo contratado': f[e[s]], 'Llegada solicitud': h[s]})
                t[m] = h[s] + dist[(m,s)] + d[s]

            print('Tiempo del médico: ', t[m])
            u[m] = (v[s][0], v[s][1])
            
        else:
            contador = contador + 1

    if contador == len(M_disponible):
        print('Minutos de la solicitud: ', h[s])
        print('Se cancela la solicitud: ', s)

    
            

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/juancbello/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/939d40f1ecbe4ebca472f72a7e44ec7a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/939d40f1ecbe4ebca472f72a7e44ec7a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 15 RHS
At line 17 BOUNDS
At line 20 ENDATA
Problem MODEL has 1 rows, 2 columns and 2 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.15666 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 3.15666 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 w

### Mostrar Gannt

In [ ]:
import plotly.express as px
import pandas as pd
#Poner los datos de la lista en un dataframe
df1 = pd.DataFrame(gantt)
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
df1['Solicitud'] = df1['Solicitud'].astype(str)
# poner la columna médico como texto
df1['Médico'] = df1['Médico'].astype(str)

fig = px.timeline(df1, x_start="Inicio", x_end="Fin", y="Médico", color="Médico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")
fig.update_layout(xaxis_title="Tiempo", yaxis_title="Médico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()

## Calcular Indicadores

- Distancia total recorrida por el médico
- Cantidad de solicitudes atendidas por cada médico
- Tiempo trabajado de cada médico
- Tiempo ocioso de cada médico
- Tiempo promedio de espera de la solicitudes atendidas por cada médico
- Cantidad de solicitudes atendidas fuera del tiempo contratado por cada médico
- Tiempo demora promedio de las solicitudes atendidas fuera del tiempo contratado por cada médico

In [5]:
from datetime import datetime

# Crear un dataframe con los datos de los doctores
res = pd.DataFrame(columns=['Medico', 'Distancia total', 'Cantidad de solicitudes', 'Tiempo trabajado', 'Tiempo ocioso', 'Tiempo de espera', 'Cantidad de solicitudes demoradas', 'Tiempo de demora', 'Tiempo en servicio', 'Tiempo extra', 'Cantidad de solicitudes por hora', 'Tiempo de espera promedio'])

# Agregar los médicos al dataframe
for i in gantt:
    if i['Medico'] not in res['Medico'].values:
        row = {'Medico': i['Medico'], 'Distancia total': 0, 'Cantidad de solicitudes': 0, 'Tiempo trabajado': 0,'Tiempo ocioso':0, 'Tiempo de espera': 0, 'Cantidad de solicitudes demoradas': 0, 'Tiempo de demora': 0, 'Tiempo en servicio': 0, 'Tiempo extra': 0, 'Cantidad de solicitudes por hora': 0, 'Tiempo de espera promedio': 0}
        res = pd.concat([res, pd.DataFrame(row, index=[0])], ignore_index=True)

# Calcular los datos de los médicos
for i in gantt:

    fin_solicitud = datetime.strptime(i['Fin'], "%Y-%m-%d %H:%M:%S")
    inicio_solicitud = datetime.strptime(i['Inicio'], "%Y-%m-%d %H:%M:%S")

    inicio_jornada = i['Inicio Jornada']
    fin_jornada = i['Fin Jornada']
    fin_traslado = i['Fin traslado']
    llegada_solicitud = i['Llegada solicitud']
    inicio_traslado = i['Inicio traslado']
    tiempo_contratado = i['Tiempo contratado']
    if fin_solicitud.date() != inicio_solicitud.date() or fin_solicitud.hour * 60 + fin_solicitud.minute < inicio_traslado:
        # Se pasa fin solicitud a minutos y se agrega 1440 minutos que son los minutos de un día
        fin_solicitud = fin_solicitud.hour * 60 + fin_solicitud.minute + 1440
    else:
        fin_solicitud = fin_solicitud.hour * 60 + fin_solicitud.minute

    if inicio_solicitud.hour * 60 + inicio_solicitud.minute < llegada_solicitud:
        inicio_solicitud = inicio_solicitud.hour * 60 + inicio_solicitud.minute + 1440 + inicio_solicitud.second / 60
    else:
        inicio_solicitud = inicio_solicitud.hour * 60 + inicio_solicitud.minute + inicio_solicitud.second / 60

    # Calcular la distancia total recorrida por cada médico
    res.loc[res['Medico'] == i['Medico'], 'Distancia total'] += i['Distancia']

    # Calcular la cantidad de solicitudes atendidas por cada médico
    res.loc[res['Medico'] == i['Medico'], 'Cantidad de solicitudes'] += 1

    # Calcular el tiempo trabajado por cada médico
    tiempo_trabajando = fin_solicitud - inicio_traslado
    res.loc[res['Medico'] == i['Medico'], 'Tiempo trabajado'] += tiempo_trabajando

    # Calcular el tiempo de espera de cada médico
    tiempo_espera = inicio_solicitud - llegada_solicitud
    res.loc[res['Medico'] == i['Medico'], 'Tiempo de espera'] += tiempo_espera

    # Calcular la cantidad de solicitudes demoradas por cada médico
    if inicio_solicitud - llegada_solicitud > tiempo_contratado:
        res.loc[res['Medico'] == i['Medico'], 'Cantidad de solicitudes demoradas'] += 1

    # Calcular el tiempo de demora de cada médico
    if inicio_solicitud - llegada_solicitud > tiempo_contratado:
        tiempo_demora = inicio_solicitud - llegada_solicitud - tiempo_contratado
        res.loc[res['Medico'] == i['Medico'], 'Tiempo de demora'] += tiempo_demora

# Para cada médico encontrar el maximo tiempo de fin solicitud
for i in res['Medico']:
    maximo = 0
    for j in gantt:
        if j['Medico'] == i:
            if j['Fin atencion']> maximo:
                maximo = j['Fin atencion']
                # Calcular el tiempo ocioso de cada médico
                maximo = maximo - j['Inicio Jornada']
    
    res.loc[res['Medico'] == i, 'Tiempo ocioso'] = maximo - res.loc[res['Medico'] == i, 'Tiempo trabajado']

# Calcular el tiempo en servicio de cada médico
res['Tiempo en servicio'] = res['Tiempo trabajado'] + res['Tiempo ocioso']

# Calcular el tiempo extra de cada médico si es negativo es 0
res['Tiempo extra'] = res['Tiempo en servicio'] - 480
res.loc[res['Tiempo extra'] < 0, 'Tiempo extra'] = 0

# Calcular la cantidad de solicitudes por hora de cada médico
res['Cantidad de solicitudes por hora'] = res['Cantidad de solicitudes'] / (res['Tiempo en servicio'] / 60)

# Calcular el tiempo de espera promedio de cada médico
res['Tiempo de espera promedio'] = res['Tiempo de espera'] / res['Cantidad de solicitudes']


In [6]:
import pandas as pd
import openpyxl

# Cargar el archivo de Excel existente
excel_file = 'resultadosv5.xlsx'
book = openpyxl.load_workbook(excel_file)

# Crear un objeto ExcelWriter que apunte al archivo existente
writer = pd.ExcelWriter(excel_file, engine='openpyxl')
writer.book = book

# Agregar una nueva hoja al archivo existente
res.to_excel(writer, sheet_name='Domingo OP')

# Guardar los cambios y cerrar el objeto ExcelWriter
writer.save()

/var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/ipykernel_49954/1827820835.py:10: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
/var/folders/m6/xx2tl5h91wbf651lwjmb3f900000gn/T/ipykernel_49954/1827820835.py:16: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


## Situación Actual

### Calcular la distancia/tiempo entre cada par de nodos

In [ ]:
from math import radians, cos, sin, asin, sqrt
def distancia_tierra(lat1, lat2, lon1, lon2):
    
    # Convertir grados a radianes.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Formula Haversine, para calcular la distancia entre dos puntos de una esfera dadas sus coordenadas de longitud y latitud
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    
    # Radio de la tierra en km.
    r = 6371
    
    respuesta = ((c * r)/24.9)*60
    # Calcular la distancia final en Km.
    return respuesta

In [ ]:
import datetime

def convertMin(minutos):
    # Obtenemos la fecha de hoy
    fecha_actual = datetime.date.today()
    # Combinamos la fecha de hoy con la hora 0:00:00
    fecha_hora = datetime.datetime.combine(fecha_actual, datetime.time.min) + datetime.timedelta(minutes=minutos)
    # Convertimos la fecha y hora a formato de cadena
    fecha_hora_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_hora_str

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

# Cargar del excel instancias la hoja Instancia1
Actual = pd.read_excel('Instancias.xlsx', sheet_name='Actual7') # Cambiar de acuerdo a la instancia

latitud_sede = 4.681230925773865
longitud_sede = -74.06200456196707

# Crear columna distancia en el dataframe
Actual['Distancia'] = 0
# Agrupar el DataFrame por la columna 'Medico'
grupos_medicos = Actual.groupby('Medico')

# Calcular la distancia total recorrida por cada médico
for i in grupos_medicos:
    # Crear una lista con las latitudes y longitudes de cada médico
    latitudes = i[1]['Latitud'].tolist()
    longitudes = i[1]['Longitud'].tolist()
    # Crear una lista con las distancias entre cada punto
    distancias_list = []
    for j in range(len(latitudes)):
        if j == 0:
            distancias_list.append(distancia_tierra(latitud_sede, latitudes[j], longitud_sede, longitudes[j]))
        else:
            distancias_list.append(distancia_tierra(latitudes[j-1], latitudes[j], longitudes[j-1], longitudes[j]))
        # Agregar la distancia a cada solicitud
        Actual.loc[Actual['Solicitud'] == i[1]['Solicitud'].tolist()[j], 'Distancia'] = distancias_list[j]

# crear un dataframe con los datos de las solicitudes
res0 = pd.DataFrame(columns=['Medico', 'Distancia total', 'Cantidad de solicitudes', 'Tiempo trabajado', 'Tiempo ocioso', 'Tiempo de espera', 'Cantidad de solicitudes demoradas', 'Tiempo de demora', 'Tiempo en servicio', 'Tiempo extra', 'Cantidad de solicitudes por hora', 'Tiempo de espera promedio'])

# Agregar las solicitudes al dataframe
for i in Actual['Medico'].unique():
    row = {'Medico': i, 'Distancia total': 0, 'Cantidad de solicitudes': 0, 'Tiempo trabajado': 0,'Tiempo ocioso':0, 'Tiempo de espera': 0, 'Cantidad de solicitudes demoradas': 0, 'Tiempo de demora': 0, 'Tiempo en servicio': 0, 'Tiempo extra': 0, 'Cantidad de solicitudes por hora': 0, 'Tiempo de espera promedio': 0}
    res0 = pd.concat([res0, pd.DataFrame(row, index=[0])], ignore_index=True)

# Calcular la distancia total recorrida por cada médico, agrupando por médico y sumando la distancia
for i, row in Actual.iterrows():
    # agrupar por médico
    res0.loc[res0['Medico'] == row['Medico'], 'Distancia total'] += row['Distancia']
    # Calcular la cantidad de solicitudes atendidas por cada médico
    res0.loc[res0['Medico'] == row['Medico'], 'Cantidad de solicitudes'] += 1
    # Calcular el tiempo de trabajo
    tiempo_trabajado = (row['Hora fin'] - row['Hora inicio']) + row['Distancia']
    # Agregar el tiempo trabajado al dataframe
    res0.loc[res0['Medico'] == row['Medico'], 'Tiempo trabajado'] += tiempo_trabajado

# Calcular el tiempo de espera de cada médico
for i, row in Actual.iterrows():
    # Calcular el tiempo de espera
    tiempo_espera = row['Hora inicio'] - row['Llegada solicitud']
    # Agregar el tiempo de espera al dataframe
    res0.loc[res0['Medico'] == row['Medico'], 'Tiempo de espera'] += tiempo_espera

#calcular la cantidad de solicitudes demoradas
for i, row in Actual.iterrows():
    # Calcular el tiempo de espera
    tiempo_espera = row['Hora inicio'] - row['Llegada solicitud']
    # Si el tiempo de espera es mayor a 15 minutos se considera demorada
    if tiempo_espera > 180:
        res0.loc[res0['Medico'] == row['Medico'], 'Cantidad de solicitudes demoradas'] += 1

# Calcular el tiempo de demora de cada médico
for i, row in Actual.iterrows():
    # Calcular el tiempo de espera
    tiempo_espera = row['Hora inicio'] - row['Llegada solicitud']
    # Si el tiempo de espera es mayor a 15 minutos se considera demorada
    if tiempo_espera > 180:
        res0.loc[res0['Medico'] == row['Medico'], 'Tiempo de demora'] += tiempo_espera

# Calcular el tiempo en servicio de cada médico es el tiempo maximo de finalizacion de una solicitud - tiempo minimo de inicio de una solicitud
for i in Actual['Medico'].unique():
    # Calcular el tiempo en servicio
    tiempo_en_servicio = Actual.loc[Actual['Medico'] == i, 'Hora fin'].max() - Actual.loc[Actual['Medico'] == i, 'Hora inicio'].min()
    # Agregar el tiempo en servicio al dataframe
    res0.loc[res0['Medico'] == i, 'Tiempo en servicio'] = tiempo_en_servicio

# Calcular el tiempo de ocio de cada médico: maximo fin de solicitud - minimo inicio de solicitud - tiempo trabajado
for i in Actual['Medico'].unique():
    res0.loc[res0['Medico'] == i, 'Tiempo ocioso'] = Actual.loc[Actual['Medico'] == i, 'Hora fin'].max() - Actual.loc[Actual['Medico'] == i, 'Hora inicio'].min() - res0.loc[res0['Medico'] == i, 'Tiempo trabajado'].values
    # si dio negativo el tiempo de ocio se pone en 0
    if res0.loc[res0['Medico'] == i, 'Tiempo ocioso'].values < 0:
        res0.loc[res0['Medico'] == i, 'Tiempo ocioso'] = 0


# Calcular el tiempo extra de cada médico es el tiempo en servicio - tiempo trabajado
for i in Actual['Medico'].unique():
    # Calcular el tiempo extra
    tiempo_extra = res0.loc[res0['Medico'] == i, 'Tiempo en servicio'].values - 480
    # Agregar el tiempo extra al dataframe
    res0.loc[res0['Medico'] == i, 'Tiempo extra'] = tiempo_extra
    # si dio negativo el tiempo extra se pone en 0
    if res0.loc[res0['Medico'] == i, 'Tiempo extra'].values < 0:
        res0.loc[res0['Medico'] == i, 'Tiempo extra'] = 0

# Calcular la cantidad de solicitudes por hora de cada médico: cantidad de solicitudes / tiempo trabajado
for i in Actual['Medico'].unique():
    # Calcular la cantidad de solicitudes por hora
    solicitudes_por_hora = res0.loc[res0['Medico'] == i, 'Cantidad de solicitudes'].values / (res0.loc[res0['Medico'] == i, 'Tiempo trabajado'].values / 60)
    # Agregar la cantidad de solicitudes por hora al dataframe
    res0.loc[res0['Medico'] == i, 'Cantidad de solicitudes por hora'] = solicitudes_por_hora

# Calcular el tiempo de espera promedio de cada médico: tiempo de espera / cantidad de solicitudes
for i in Actual['Medico'].unique():
    # Calcular el tiempo de espera promedio
    tiempo_espera_promedio = res0.loc[res0['Medico'] == i, 'Tiempo de espera'].values / res0.loc[res0['Medico'] == i, 'Cantidad de solicitudes'].values
    # Agregar el tiempo de espera promedio al dataframe
    res0.loc[res0['Medico'] == i, 'Tiempo de espera promedio'] = tiempo_espera_promedio

In [ ]:
import pandas as pd
import openpyxl

# Cargar el archivo de Excel existente
excel_file = 'resultados.xlsx'
book = openpyxl.load_workbook(excel_file)

# Crear un objeto ExcelWriter que apunte al archivo existente
writer = pd.ExcelWriter(excel_file, engine='openpyxl') 
writer.book = book

# Agregar una nueva hoja al archivo existente
res0.to_excel(writer, sheet_name='Domingo AC')

# Guardar los cambios y cerrar el objeto ExcelWriter
writer.save()

In [ ]:
# poner los datos en el formato que requiere plotly poner la columna solicitud como texto
Actual['Solicitud'] = Actual['Solicitud'].astype(str)
# poner la columna médico como texto
Actual['Medico'] = Actual['Medico'].astype(str)

for index, i in enumerate(Actual['Hora inicio']):
    nuevo = convertMin(i)
    Actual.loc[index, 'Hora inicio'] = nuevo

for index, i in enumerate(Actual['Hora fin']):
    nuevo = convertMin(i)
    Actual.loc[index, 'Hora fin'] = nuevo

for index, i in enumerate(Actual['Hora programada']):
    nuevo = convertMin(i)
    Actual.loc[index, 'Hora programada'] = nuevo

# ordenar dataframe por hora de inicio
Actual = Actual.sort_values(by=['Hora inicio'])

fig = px.timeline(Actual, x_start="Hora inicio", x_end="Hora fin", y="Medico", color="Medico",
                  hover_name="Solicitud",
                  title="Solicitudes atendidas por médicos")
fig.update_layout(xaxis_title="Tiempo", yaxis_title="Medico")
fig.update_yaxes(autorange="reversed")
fig.update_xaxes(rangeslider_visible=True)
# Hacer el grafico más grande para que se vea mejor
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
)
fig.show()